### **STEP-1**. AI Solution 및 Instance 등록을 위한 준비 작업
&#x1F600; **등록 할 AI Contents 의 experimental_plan.yaml 를 alo/config/ 에 준비해 둡니다.**

&#x1F600; **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     

1. ALO 의 main.py 파일이 존재하는 위치에서 아래 명령어들을 순차 실행합니다.
> conda create -n {ENV-NAME} python=3.10 \
> conda init bash \
> conda activate {ENV-NAME} \
> python main.py \
> pip install ipykernel \
> pip install requests \
> python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}

2. 본 jupyter notebook 에서, 위에서 생성한 ipykernel 을 선택 합니다. \
   가령 tcr이라는 이름의 가상환경을 만들었다면, 아래와 같이 선택합니다.

<div style="margin: 40px">
<img src="./image/ipykernel.png" width="400">
</div>


&#x1F600; **아래 STEP들을 하나씩 실행시키면서, <u>< 사용자 입력 ></u>이라고 주석 표기된 내용을 적절히 변경해주세요.**     

----

### **STEP-2**. AI Solution 이름 선택     

#### **STEP-2-1**. AI Conductor 시스템 URI 셋팅
&#x1F600; 로그인 요청 및 시스템 담당으로부터 사용 가능한 시스템 URI를 확인합니다. 
- 고객지수플랫폼 Development 
> URI: "https://aic-kic.aidxlge.com/"
- 담당서버 테스트 환경       
> URI = "http://10.158.2.243:9999/" 
- 사외 테스트 환경 (LDAP 로그인 불가)
> URI = "https://web.aic-dev.lgebigdata.com/" 

> 시스템 URI 및 로그인 정보, ECR TAG, ICON FILE 명 등 사용자 입력부

In [43]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
# user_input ={
#     # 로그인 정보: EP 정보로 입력해주세요
#     'login_id': "cism-dev", #"magna-dev", #'ws.jang', # "cism-dev"
#     'login_pw': "cism-dev@com", #'magna-dev@com', # "cism-dev
#     'solution_type': "single", # "single", "multi"
#     'model_path': "", ## artifact 존재 시, 자동 steup, 없으면 pass. 기록하면 1순위 우선순위  
#     'update': True,
#     'description': {
#         'title': "UI solution title",
#     },
#     'contents_type': {
#             'support_labeling': False,
#             'inference_result_datatype': 'table', # 'image'
#             'train_datatype': 'table' # 'image'
#     },
# }

# user_input ={
#     # 로그인 정보: EP 정보로 입력해주세요
#     'login_id': "cism-dev", #"magna-dev", #'ws.jang', # "cism-dev"
#     'login_pw': "cism-dev@com", #'magna-dev@com', # "cism-dev
#     'solution_name': "test_12346",
#     'solution_type': "single", # "single", "multi"
#     'model_path': "", ## artifact 존재 시, 자동 steup, 없으면 pass. 기록하면 1순위 우선순위  
#     'description': {
#         'title': "UI solution title",
#         'overview': "AI Advisor Test",
#         'input_data': "Test input s3 bucket",
#         'output_data': "Test output s3 bucket",
#         'user_parameters': "Test params",
#         'algorithm': "ALO"
#     },
#     'train_resource': "standard",  ## low, standard, high
#     'contents_type': {
#             'support_labeling': False,
#             'inference_result_datatype': 'table', # 'image'
#             'train_datatype': 'table' # 'image'
#     },
#     'icon_name': 'ic_bolt_fastening_inspection'
}


ds

TODO 컨텐츠 1회 실행하면 input folder 에서 데이터 가져옴을 명시하기 



In [134]:
import sys
from IPython.display import SVG, display, HTML
import os 


DEBUG_MODE = True

try:
    del sys.modules['register_utils_ssh'] 
except:
    pass
from register_utils_ssh import SolutionRegister

register = SolutionRegister()
register.debugging = False

## 로그인
register.login(user_input["login_id"], user_input["login_pw"])


#####################
### Cell 나누기 -- Solution Name 입력
#####################
# AI Solution 이름 설정 (기존에 존재하는 중복 이름 허용X)

if not user_input["solution_name"]:
    # soltuion 이름 입력 받기 
    display(HTML('<p style="font-size: 20px;"> < Create the name of AI Solution! > </p>'))
    name = input("- Enter the name you want to create: ")
else:
    name = user_input["solution_name"]

register.check_solution_name(name)

# check workspace (ECR, S3정보 셋팅까지 진행)
register.load_system_resource() 
# 받아온 workspace 정보 기반으로 solution_metadata.yaml 셋팅 
register._init_solution_metadata()



#####################
### Cell 나누기 -- description 추가 
#####################
register.set_description(user_input["description"])

# wrangler 정보 등록 
register.set_wrangler()

#####################
### Cell 나누기 -- contents type 추가 
#####################

register.set_edge(user_input["contents_type"])


#####################
### Cell 나누기 -- icon 설명
#####################
# UI에 표시될 아이콘 이미지를 s3에 업로드합니다. 
html_content = register.s3_upload_icon_display()
display(HTML(html_content))  ## 테스트중에는 꺼둠
register.s3_upload_icon(name=user_input["icon_name"])


#####################
### Cell 나누기 -- train resource 설정
#####################
## common
register._s3_access_check()  ## s3 instnace 생성 

register._sm_append_pipeline(pipeline_name='train')
register.set_resource_display()
register.set_resource(user_input['train_resource'])  ## resource 선택은 spec-out 됨

#####################
### Cell 나누기 -- user parameters 설정 (train, inference 한번에)
#####################
candidate = register.set_candidate_parameters()

#####################
### Cell 나누기 -- Set Train 
#####################

####### Train pipeline 세팅  ########
register.s3_upload_data()
register.s3_upload_artifacts()
## train docker container 를 ecr 에 업로드
if not DEBUG_MODE:
    register.make_docker_container()

####### Inference pipeline 세팅  ########
register._sm_append_pipeline(pipeline_name='inference')
register.set_resource(resource='standard')  ## resource 선택은 spec-out 됨
register.set_candidate_parameters(display_table=False)

register.s3_upload_data()
register.s3_upload_artifacts()  ## inference 시, upload model 도 진행
if not DEBUG_MODE:
    register.make_docker_container()


## finally
# register.register_solution()


#########################################
#######    Initiate ALO operation mode
#########################################

[SYSTEM] Solutoin 등록에 필요한 setup file 들을 load 합니다. 
Infra setup 파일이 존재 하지 않으므로, Default 파일을 load 합니다. (path: ./infra_setup.yaml)
{'AIC_URI': 'https://web.aic-dev.lgebigdata.com/',
 'BUILDAH_TAGS': ['Key=Company,Value=LGE',
                  'Key=Owner,Value=IC360',
                  'Key=HQ,Value=CDO',
                  'Key=Division,Value=CDO',
                  'Key=Infra Region,Value=KIC',
                  'Key=Service Mode,Value=OP',
                  'Key=Cost Type,Value=COMPUTING',
                  'Key=Project,Value=CIS',
                  'Key=Sub Project,Value=CISM',
                  'Key=System,Value=AIDX'],
 'BUILD_METHOD': 'docker',
 'ECR_TAG': 'latest',
 'INFERENCE_RESOURCE': 'amd',
 'LOGIN_MODE': 'static',
 'REGION': 'ap-northeast-2',
 'SOLUTION_TYPE': 'private',
 'VERSION': 1.0,
 'WORKSPACE_NAME': 'cism-ws'}
API setup 파일이 존재 하지 않으므로, Default 파일을 

해당 계정으로 접근 가능한 workspace list: ['cism-ws']
[SYSTEM] 접근 요청하신 workspace (cism-ws) 은 해당 계정으로 접근 가능합니다.

#########################################
#######    Solution Name Creation
#########################################

[SUCCESS] 입력하신 Solution Name (test_12346)은 사용 가능합니다. 
[SYSTEM] Solution Name List (in-use):
╒════╤══════════════════════════╕
│    │ AI solutions             │
╞════╪══════════════════════════╡
│  0 │ solution-hyunsoo-cism-20 │
├────┼──────────────────────────┤
│  1 │ pytest-solution-1        │
├────┼──────────────────────────┤
│  2 │ solution-hyunsoo-cism-2  │
├────┼──────────────────────────┤
│  3 │ solution-hyunsoo-cism-1  │
╘════╧══════════════════════════╛

#########################################
#######    Check ECR & S3 Resource
#########################################

[SYSTEM] AWS ECR:  
086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/cism/
[SYSTEM] AWS S3 buckeet:  
s3-an2-hyunsoo-dev-cism

###########################


#########################################
#######    Upload icon
#########################################

[SUCCESS] update solution_metadata.yaml:
description: -icon: s3://s3-an2-hyunsoo-dev-aia/icons/ic_bolt_fastening_inspection.svg 

#########################################
#######    Check to access S3
#########################################

[INFO] AWS region: ap-northeast-2
[INFO] AWS S3 access check: OK

#########################################
#######    Display train Resource List
#########################################

train 에 사용될 resource 를 선택해주세요(type['high', 'standard']):
╒════╤══════════╤══════════════════╤════════╤══════════╤═══════╕
│    │ name     │ label            │   vcpu │   ram_gb │   gpu │
╞════╪══════════╪══════════════════╪════════╪══════════╪═══════╡
│  0 │ high     │ cism-ws-high     │      8 │       32 │     0 │
├────┼──────────┼──────────────────┼────────┼──────────┼───────┤
│  1 │ standard │ cism-ws-standard │      2 │        8 │     0 │
╘════╧═══

In [129]:
from pprint import pprint
pprint(candidate)

{'inference_pipeline': [{'args': [{'description': '',
                                   'name': 'concat_dataframes',
                                   'type': ''},
                                  {'description': '',
                                   'name': 'drop_columns',
                                   'type': ''},
                                  {'description': '',
                                   'name': 'encoding',
                                   'type': ''},
                                  {'description': '',
                                   'name': 'groupkey_columns',
                                   'type': ''},
                                  {'description': '',
                                   'name': 'input_path',
                                   'type': ''},
                                  {'description': '',
                                   'name': 'time_column',
                                   'type': ''},
                                

In [80]:
import pandas as pd
register._read_experimentalplan_yaml(register.exp_yaml_path)
params = register.exp_yaml['user_parameters'].copy()


# 컬럼 이름 설정
columns = ['pipeline', 'step', 'parmeter', 'value']
df = pd.DataFrame(columns=columns)

print(params)
idx = 0
for pipe_dict in params:
    if 'train_pipeline' in list(pipe_dict.keys()):
        pipe_name = 'train'
    elif 'inference_pipeline' in list(pipe_dict.keys()):
        pipe_name = 'inference'
    else:
        pipe_name = None

    for step_dict in pipe_dict[f'{pipe_name}_pipeline']:
        step_name = step_dict['step']
        for key, value in step_dict['args'][0].items():

            item = [pipe_name, step_name, key, value]
            df.loc[idx] = item

            idx += 1
            

MAX_LEN = 40
df['value'] = df['value'].astype(str)
df['value'] = df['value'].apply(lambda x: x[:MAX_LEN] + '...' if len(x) > MAX_LEN else x)
# print(df.to_markdown(tablefmt='fancy_grid'))

print(register.candidate_params)


[{'train_pipeline': [{'args': [{'concat_dataframes': True, 'drop_columns': None, 'encoding': None, 'groupkey_columns': None, 'input_path': 'default_train', 'time_column': None, 'use_all_x': False, 'x_columns': ['input_x0', 'input_x1', 'input_x2', 'input_x3'], 'y_column': 'target'}], 'step': 'input'}, {'args': [{'custom': {}, 'mode': 'auto'}], 'step': 'preprocess'}, {'args': [{'ignore_label_class': None, 'label_sampling': False, 'label_sampling_num': None, 'label_sampling_num_type': None, 'negative_target_class': None, 'sampling_groupkey_columns': None, 'sampling_method': 'random', 'sampling_num': 0.8, 'sampling_num_type': 'ratio', 'sampling_type': 'none'}], 'step': 'sampling'}, {'args': [{'data_split_method': 'cross_validate', 'evaluation_metric': 'accuracy', 'model_list': ['lgb', 'rf', 'cb'], 'model_type': 'classification', 'num_hpo': 3, 'param_range': {'cb': {'max_depth': [5, 9], 'n_estimators': [100, 500]}, 'gbm': {'max_depth': [5, 7], 'n_estimators': [300, 500]}, 'lgb': {'max_depth

TODO 진행할 사항 아래에 남겨두기 

In [ ]:
## Extra Job 

registerer.register_solution_instance()
registerer.register_stream()
registerer.request_run_stream()
registerer.get_stream_status()
registerer.download_artifacts()

# 가상환경이 잘 connected 돼있는지 확인 
!which python 
# 현재 작업경로 확인 
!pwd

# s3로부터 다운로드받은 train artifacts를 scripts 폴더 상위 경로의 main.py랑 같은 위치로 옮기고 추론 실행 
import os
os.makedirs("./.train_artifacts",  exist_ok=True)

!tar -xvf ./train_artifacts.tar.gz -C ./.train_artifacts/
!cp -r .train_artifacts ../../
!rm -rf ./.train_artifacts

!python ../../main.py --mode inference